Dataset Builder
-----------------------------------------

In [2]:
import gc
import os

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [3]:
# reduce memory usage by converting to proper data types
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024 ** 2
    print("Memory usage of dataframe is {:.2f} MB".format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype("category")

    end_mem = df.memory_usage().sum() / 1024 ** 2
    print("Memory usage after optimization is: {:.2f} MB".format(end_mem))
    print("Decreased by {:.1f}%".format(100 * (start_mem - end_mem) / start_mem))

    return df

## DATA DESCRIPTION

### Temporals

These are the time dependent features in for every lat/lon combination (which determines a grid cell)

Most variables at time t means the average of the earlier 3 hours. SPEEDMAX is the max of the earlier 3 hr, PRECTOT is the sum of the earlier 3 hr, and fire_count is the interpolated estimation of the fire_count at time t.

detailes: https://docs.google.com/spreadsheets/d/1JvtprfdTgBXrQ9zVaWjCtBl2QDmq8YulS4_YfUK7MC8/edit#gid=0

T2M 2-meter_air_temperature<br>
U2M 2-meter_eastward_wind<br>
V2M 2-meter_northward_wind<br>
QV2M 2-meter_specific_humidity<br>
T10M 10-meter_air_temperature<br>
U10M 10-meter_eastward_wind<br>
V10M 10-meter_northward_wind<br>
QV10M 10-meter_specific_humidity<br>
U50M eastward_wind_at_50_meters<br>
V50M northward_wind_at_50_meters<br>
SLP sea_level_pressure<br>
PRECTOT total_precipitation<br>
SPEEDMAX surface_wind_speed max<br>
BSTAR surface_bouyancy_scale<br>
GRN greeness_fraction<br>
LAI leaf_area_index<br>
GWETROOT root_zone_soil_wetness<br>
GWETTOP surface_soil_wetness<br>
TSURF surface_temperature_of_land_incl_snow<br>

U2M-V2M, U10M-V10M, U50M-V50M must be used together, because they determine the wind direction and speed together.

### Constants

This dataset containes the time independent features. We have one row for for each cells, no time steps.

detailes https://docs.google.com/spreadsheets/d/1JvtprfdTgBXrQ9zVaWjCtBl2QDmq8YulS4_YfUK7MC8/edit#gid=0

SGH isotropic stdv of GWD topography<br>
elevation elevation<br>
population density<br>

Numbers from 10 to 220 Land Cover features, the given land cover feature as percent of cell cover.

10 Cropland, rainfed<br>
11 Herbaceous cover<br>
12 Tree or shrub cover<br>
30 Mosaic cropland (>50%) / natural vegetation (tree, shrub, herbaceous cover) (<50%)<br>
40 Mosaic natural vegetation (tree, shrub, herbaceous cover) (>50%) / cropland (<50%)<br>
60 Tree cover, broadleaved, deciduous, closed to open (>15%)<br>
70 Tree cover, needleleaved, evergreen, closed to open (>15%)<br>
80 Tree cover, needleleaved, deciduous, closed to open (>15%)<br>
90 Tree cover, mixed leaf type (broadleaved and needleleaved)<br>
100 Mosaic tree and shrub (>50%) / herbaceous cover (<50%)<br>
110 Mosaic herbaceous cover (>50%) / tree and shrub (<50%)<br>
120 Shrubland<br>
130 Grassland<br>
150 Sparse vegetation (tree, shrub, herbaceous cover) (<15%)<br>
160 Tree cover, flooded, fresh or brakish water<br>
180 Shrub or herbaceous cover, flooded, fresh/saline/brakish water<br>
190 Urban areas<br>
200 Bare areas<br>
210 Water bodies<br>
220 Permanent snow and ice<br>

Numbers from 1 to 9 broader merged Land Cover features, the given land cover feature as percent of cell cover.

1 agriculture<br>
2 forest<br>
3 grassland<br>
4 wetland<br>
5 settlement<br>
6 shrubland<br>
7 sparse vegetation<br>
8 bare areas<br>
9 water<br>

max_type_freq_detailed - most frequent detailed feature type cover percent in grid cell max_type_detailed - most frequent detailed feature type in grid cell max_type_freq_merged - most frequent merged feature type cover percent in grid cell max_type_merged - most frequent nerged feature type in grid cell

### Other
EVI	MIR	EVI_wasNA	MIR_wasNA	dayOfWeek	dayOfYear	sin_dayOfYear	cos_dayOfYear	isWeekend

In [4]:
os.listdir("../input")
dataDir = "../input/"

FileNotFoundError: [Errno 2] No such file or directory: '../input'

In [4]:
print(os.listdir("../input/"))

['fire-grid-pivot-check', 'ibm-fire-build-the-constants-csv', 'temporal']


In [5]:
dtype_dict = {
    "lat": np.float16,
    "lon": np.float16,
    "U2M": np.float16,
    "T2M": np.float16,
    "V50M": np.float16,
    "T10M": np.float16,
    "SLP": np.float32,
    "V2M": np.float16,
    "V10M": np.float16,
    "U50M": np.float16,
    "U10M": np.float16,
    "QV2M": np.float16,
    "QV10M": np.float16,
    "SPEEDMAX": np.float16,
    "BSTAR": np.float16,
    "PRECTOT": np.float16,
    "GWETTOP": np.float16,
    "LAI": np.float32,
    "GWETROOT": np.float32,
    "GRN": np.float32,
    "TSURF": np.float32,
    "EVI": np.float32,
    "MIR": np.float32,
    "EVI_wasNA": np.bool,
    "MIR_wasNA": np.bool,
    "fire_count": np.float16,
    "dayOfYear": np.int16,
    "dayOfWeek": np.int8,
}

In [6]:
df = pd.read_csv(
    dataDir + "/temporal/fire_merra_VI_embs_3h_basegrid_divider_1.csv",
    usecols=[
        "timestamp",
        "lat",
        "lon",
        "U2M",
        "T2M",
        "V50M",
        "T10M",
        "SLP",
        "V2M",
        "V10M",
        "U50M",
        "U10M",
        "QV2M",
        "QV10M",
        "SPEEDMAX",
        "BSTAR",
        "PRECTOT",
        "GWETTOP",
        "LAI",
        "GWETROOT",
        "GRN",
        "TSURF",
        "fire_count",
        "EVI",
        "MIR",
        "EVI_wasNA",
        "MIR_wasNA",
        "dayOfYear",
        "dayOfWeek",
    ],
    dtype=dtype_dict,
)

In [7]:
df.shape

(9379818, 29)

In [8]:
df = reduce_mem_usage(df)

Memory usage of dataframe is 670.90 MB
Memory usage after optimization is: 600.79 MB
Decreased by 10.5%


In [9]:
df.head(2)

,timestamp,lat,lon,U2M,T2M,V50M,T10M,SLP,V2M,V10M,U50M,U10M,QV2M,QV10M,SPEEDMAX,BSTAR,PRECTOT,GWETTOP,LAI,GWETROOT,GRN,TSURF,fire_count,EVI,MIR,EVI_wasNA,MIR_wasNA,dayOfWeek,dayOfYear
0,2010-01-02 18:00:00,32.0,-123.750,0.478271,289.00,-7.226562,289.00,102313.367188,-5.574219,-6.488281,0.542969,0.557129,0.010506,0.010414,8.273438,0.000602,0.0,1.0,1.000000e+15,1.000000e+15,1.000000e+15,1.000000e+15,0.0,-0.099976,0.799805,1.0,1.0,5,2
1,2010-01-02 18:00:00,32.0,-123.125,0.916016,288.75,-7.765625,288.75,102254.703125,-5.902344,-6.914062,1.139648,1.074219,0.010384,0.010300,8.734375,0.000415,0.0,1.0,1.000000e+15,1.000000e+15,1.000000e+15,1.000000e+15,0.0,-0.099976,0.799805,1.0,1.0,5,2


In [10]:
df.memory_usage().sum() / 1024**2

600.785062789917

In [11]:
# ire data is only from 32.5, so drop the lat32
df = df.loc[df["lat"] > 32]

## Dealing with missing values over the ocean

In [12]:
# LAI can be filled with zeros where NaN value, as this is over ocean
df.loc[df["LAI"] > 1000000, "LAI"] = 0

In [13]:
# GWETROOT can be filled with ones where NaN value, as this is over ocean
df.loc[df["GWETROOT"] > 1000000, "GWETROOT"] = 1

In [14]:
# GRN can be filled with zeross where NaN value, as this is over ocean
df.loc[df["GRN"] > 1000000, "GRN"] = 0

In [15]:
# NaN TSURF value is very diificult to fill with reasonable number. Will be the average of TSURF
df.loc[df["TSURF"] > 1000000, "TSURF"] = np.nan
df.loc[df["TSURF"].isnull(), "TSURF"] = df["TSURF"].mean()

## Separate no fire from fire

In [16]:
# this is really only a number which seems good separator, or at least better than zero, needs experimneting
df.loc[df["fire_count"] <= 0, "fire_count"] = -20

In [17]:
df["fire_count"].describe()

count    8933160.0
mean           NaN
std            0.0
min          -20.0
25%          -20.0
50%          -20.0
75%          -20.0
max          167.0
Name: fire_count, dtype: float64

## Embeding time

In [18]:
# day of the year to cyclic format
df["sin_dayOfYear"] = np.sin(2 * np.pi * df["dayOfYear"] / 365)
df["cos_dayOfYear"] = np.cos(2 * np.pi * df["dayOfYear"] / 365)

In [19]:
df["isWeekend"] = 0  # Initialize the column with default value of 0
df.loc[
    df["dayOfWeek"].isin([5, 6]), "isWeekend"
] = 1  # 5 and 6 correspond to Sat and Sun

In [20]:
df.head()

,timestamp,lat,lon,U2M,T2M,V50M,T10M,SLP,V2M,V10M,U50M,U10M,QV2M,QV10M,SPEEDMAX,BSTAR,PRECTOT,GWETTOP,LAI,GWETROOT,GRN,TSURF,fire_count,EVI,MIR,EVI_wasNA,MIR_wasNA,dayOfWeek,dayOfYear,sin_dayOfYear,cos_dayOfYear,isWeekend
17,2010-01-02 18:00:00,32.5,-123.750,0.739746,288.75,-7.375000,288.75,102335.367188,-5.742188,-6.671875,0.875488,0.859375,0.010490,0.010391,8.234375,0.000881,0.0,1.0,0.0,1.0,0.0,270.60733,-20.0,-0.099976,0.799805,1.0,1.0,5,2,0.034422,0.999407,1
18,2010-01-02 18:00:00,32.5,-123.125,1.321289,288.50,-7.867188,288.50,102276.703125,-6.078125,-7.101562,1.647461,1.543945,0.010391,0.010300,8.734375,0.000792,0.0,1.0,0.0,1.0,0.0,270.60733,-20.0,-0.099976,0.799805,1.0,1.0,5,2,0.034422,0.999407,1
19,2010-01-02 18:00:00,32.5,-122.500,1.771484,288.50,-8.289062,288.50,102210.703125,-6.335938,-7.437500,2.251953,2.080078,0.010292,0.010201,9.187500,0.000686,0.0,1.0,0.0,1.0,0.0,270.60733,-20.0,-0.099976,0.799805,1.0,1.0,5,2,0.034422,0.999407,1
20,2010-01-02 18:00:00,32.5,-121.875,2.291016,288.25,-8.539062,288.25,102149.703125,-6.261719,-7.414062,3.062500,2.710938,0.009834,0.009743,9.468750,0.000095,0.0,1.0,0.0,1.0,0.0,270.60733,-20.0,-0.099976,0.799805,1.0,1.0,5,2,0.034422,0.999407,1
21,2010-01-02 18:00:00,32.5,-121.250,2.873047,287.75,-8.523438,288.00,102078.367188,-6.289062,-7.453125,3.839844,3.402344,0.009514,0.009407,9.773438,0.000083,0.0,1.0,0.0,1.0,0.0,270.60733,-20.0,-0.099976,0.799805,1.0,1.0,5,2,0.034422,0.999407,1


In [21]:
df.memory_usage().sum() / 1024 ** 2

844.863655090332

## Concat the constants

In [22]:
dtype_dict = {
    "population density": np.float16,
    "SGH": np.float16,
    "lat": np.float16,
    "lon": np.float16,
    "elevation": np.float16,
    "10": np.float16,
    "11": np.float16,
    "30": np.float16,
    "40": np.float16,
    "60": np.float16,
    "70": np.float16,
    "80": np.float16,
    "90": np.float16,
    "100": np.float16,
    "110": np.float16,
    "120": np.float16,
    "130": np.float16,
    "150": np.float16,
    "160": np.float16,
    "180": np.float16,
    "190": np.float16,
    "200": np.float16,
    "210": np.float16,
    "220": np.float16,
    "agriculture": np.float16,
    "forest": np.float16,
    "grassland": np.float16,
    "wetland": np.float16,
    "settlement": np.float16,
    "shrubland": np.float16,
    "sparse_veg": np.float16,
    "bare_area": np.float16,
    "water": np.float16,
}

In [23]:
dfc = pd.read_csv(
    dataDir + "/ibm-fire-build-the-constants-csv/constants_basegrid_divider_1.csv",
    usecols=[
        "population density",
        "SGH",
        "lat",
        "lon",
        "elevation",
        "10",
        "11",
        "30",
        "40",
        "60",
        "70",
        "80",
        "90",
        "100",
        "110",
        "120",
        "130",
        "150",
        "160",
        "180",
        "190",
        "200",
        "210",
        "220",
        "agriculture",
        "forest",
        "grassland",
        "wetland",
        "settlement",
        "shrubland",
        "sparse_veg",
        "bare_area",
        "water",
    ],
    dtype=dtype_dict,
)

In [24]:
dfc = reduce_mem_usage(dfc)

Memory usage of dataframe is 0.02 MB
Memory usage after optimization is: 0.02 MB
Decreased by 0.0%


In [25]:
dfc.head()

,population density,SGH,lat,lon,elevation,10,11,30,40,60,70,80,90,100,110,120,130,150,160,180,190,200,210,220,agriculture,forest,grassland,wetland,settlement,shrubland,sparse_veg,bare_area,water
0,0.0,0.0,32.0,-123.750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,32.0,-123.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,32.0,-122.500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,32.0,-121.875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,32.0,-121.250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [26]:
df = df.merge(dfc, on=["lat", "lon"], how="left")

In [27]:
df.memory_usage().sum() / 1024 ** 2

1373.0618209838867

In [28]:
df.head()

,timestamp,lat,lon,U2M,T2M,V50M,T10M,SLP,V2M,V10M,U50M,U10M,QV2M,QV10M,SPEEDMAX,BSTAR,PRECTOT,GWETTOP,LAI,GWETROOT,GRN,TSURF,fire_count,EVI,MIR,EVI_wasNA,MIR_wasNA,dayOfWeek,dayOfYear,sin_dayOfYear,cos_dayOfYear,isWeekend,population density,SGH,elevation,10,11,30,40,60,70,80,90,100,110,120,130,150,160,180,190,200,210,220,agriculture,forest,grassland,wetland,settlement,shrubland,sparse_veg,bare_area,water
0,2010-01-02 18:00:00,32.5,-123.750,0.739746,288.75,-7.375000,288.75,102335.367188,-5.742188,-6.671875,0.875488,0.859375,0.010490,0.010391,8.234375,0.000881,0.0,1.0,0.0,1.0,0.0,270.60733,-20.0,-0.099976,0.799805,1.0,1.0,5,2,0.034422,0.999407,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2010-01-02 18:00:00,32.5,-123.125,1.321289,288.50,-7.867188,288.50,102276.703125,-6.078125,-7.101562,1.647461,1.543945,0.010391,0.010300,8.734375,0.000792,0.0,1.0,0.0,1.0,0.0,270.60733,-20.0,-0.099976,0.799805,1.0,1.0,5,2,0.034422,0.999407,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,2010-01-02 18:00:00,32.5,-122.500,1.771484,288.50,-8.289062,288.50,102210.703125,-6.335938,-7.437500,2.251953,2.080078,0.010292,0.010201,9.187500,0.000686,0.0,1.0,0.0,1.0,0.0,270.60733,-20.0,-0.099976,0.799805,1.0,1.0,5,2,0.034422,0.999407,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,2010-01-02 18:00:00,32.5,-121.875,2.291016,288.25,-8.539062,288.25,102149.703125,-6.261719,-7.414062,3.062500,2.710938,0.009834,0.009743,9.468750,0.000095,0.0,1.0,0.0,1.0,0.0,270.60733,-20.0,-0.099976,0.799805,1.0,1.0,5,2,0.034422,0.999407,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,2010-01-02 18:00:00,32.5,-121.250,2.873047,287.75,-8.523438,288.00,102078.367188,-6.289062,-7.453125,3.839844,3.402344,0.009514,0.009407,9.773438,0.000083,0.0,1.0,0.0,1.0,0.0,270.60733,-20.0,-0.099976,0.799805,1.0,1.0,5,2,0.034422,0.999407,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [29]:
# df.to_csv('basegrid_3h.csv', index=False)

In [30]:
del dfc
gc.collect()

432

## Other timeframes

In [31]:
# convert time to dateime, it is easier to handle
df["timestamp"] = pd.to_datetime(df["timestamp"])

In [32]:
# set the groups for aggregation
# 'timestamp', 'lat', 'lon' wont be aggregated, as they are the keys, but needed for selection
to_mean = [
    "timestamp",
    "lat",
    "lon",
    "U2M",
    "T2M",
    "V50M",
    "T10M",
    "SLP",
    "V2M",
    "V10M",
    "U50M",
    "U10M",
    "QV2M",
    "QV10M",
    "BSTAR",
    "GWETTOP",
    "LAI",
    "GWETROOT",
    "GRN",
    "TSURF",
    "EVI",
    "MIR",
    "population density",
    "SGH",
    "elevation",
    "10",
    "11",
    "30",
    "40",
    "60",
    "70",
    "80",
    "90",
    "100",
    "110",
    "120",
    "130",
    "150",
    "160",
    "180",
    "190",
    "200",
    "210",
    "220",
    "agriculture",
    "forest",
    "grassland",
    "wetland",
    "settlement",
    "shrubland",
    "sparse_veg",
    "bare_area",
    "water",
]
to_max = ["timestamp", "lat", "lon", "SPEEDMAX", "fire_count"]
to_sum = ["timestamp", "lat", "lon", "PRECTOT"]
to_last = [
    "timestamp",
    "lat",
    "lon",
    "dayOfWeek",
    "dayOfYear",
    "sin_dayOfYear",
    "cos_dayOfYear",
    "isWeekend",
]

### 6hr dataset

In [33]:
d6_mean = (
    df[to_mean].groupby([pd.Grouper(freq="6H", key="timestamp"), "lat", "lon"]).mean()
)
d6_sum = (
    df[to_sum].groupby([pd.Grouper(freq="6H", key="timestamp"), "lat", "lon"]).sum()
)
d6_max = (
    df[to_max].groupby([pd.Grouper(freq="6H", key="timestamp"), "lat", "lon"]).max()
)
d6_last = (
    df[to_last].groupby([pd.Grouper(freq="6H", key="timestamp"), "lat", "lon"]).last()
)

In [34]:
d6 = d6_mean.merge(d6_sum, on=["timestamp", "lat", "lon"])
d6 = d6.merge(d6_max, on=["timestamp", "lat", "lon"])
d6 = d6.merge(d6_last, on=["timestamp", "lat", "lon"])

In [35]:
d6.reset_index(inplace=True)
d6.head()

,timestamp,lat,lon,U2M,T2M,V50M,T10M,SLP,V2M,V10M,U50M,U10M,QV2M,QV10M,BSTAR,GWETTOP,LAI,GWETROOT,GRN,TSURF,EVI,MIR,population density,SGH,elevation,10,11,30,40,60,70,80,90,100,110,120,130,150,160,180,190,200,210,220,agriculture,forest,grassland,wetland,settlement,shrubland,sparse_veg,bare_area,water,PRECTOT,SPEEDMAX,fire_count,dayOfWeek,dayOfYear,sin_dayOfYear,cos_dayOfYear,isWeekend
0,2010-01-02 18:00:00,32.5,-123.750,0.480957,288.75,-7.824219,288.75,102250.843750,-6.039062,-7.039062,0.551758,0.559570,0.010353,0.010239,0.000886,1.0,0.0,1.0,0.0,270.60733,-0.099976,0.799805,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,8.492188,-20.0,5,2,0.034422,0.999407,1
1,2010-01-02 18:00:00,32.5,-123.125,1.187500,288.50,-8.328125,288.50,102198.015625,-6.367188,-7.457031,1.476562,1.390625,0.010269,0.010162,0.000800,1.0,0.0,1.0,0.0,270.60733,-0.099976,0.799805,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,9.210938,-20.0,5,2,0.034422,0.999407,1
2,2010-01-02 18:00:00,32.5,-122.500,1.733398,288.50,-8.648438,288.50,102133.843750,-6.562500,-7.714844,2.195312,2.037109,0.010193,0.010086,0.000740,1.0,0.0,1.0,0.0,270.60733,-0.099976,0.799805,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,9.695312,-20.0,5,2,0.034422,0.999407,1
3,2010-01-02 18:00:00,32.5,-121.875,2.378906,288.00,-8.640625,288.00,102077.015625,-6.351562,-7.515625,3.164062,2.812500,0.009842,0.009766,0.000181,1.0,0.0,1.0,0.0,270.60733,-0.099976,0.799805,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,9.640625,-20.0,5,2,0.034422,0.999407,1
4,2010-01-02 18:00:00,32.5,-121.250,2.876953,287.75,-8.437500,288.00,101999.679688,-6.234375,-7.382812,3.828125,3.404297,0.009506,0.009399,0.000130,1.0,0.0,1.0,0.0,270.60733,-0.099976,0.799805,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,9.773438,-20.0,5,2,0.034422,0.999407,1


In [36]:
d6.to_csv("basegrid_6h.csv", index=False)

In [37]:
# delete df insted of d6 to enable kernel release much more memory
del df
gc.collect()

7

### 12hr dataset

In [38]:
d12_mean = (
    d6[to_mean].groupby([pd.Grouper(freq="12H", key="timestamp"), "lat", "lon"]).mean()
)
d12_sum = (
    d6[to_sum].groupby([pd.Grouper(freq="12H", key="timestamp"), "lat", "lon"]).sum()
)
d12_max = (
    d6[to_max].groupby([pd.Grouper(freq="12H", key="timestamp"), "lat", "lon"]).max()
)
d12_last = (
    d6[to_last].groupby([pd.Grouper(freq="12H", key="timestamp"), "lat", "lon"]).last()
)

In [39]:
d12 = d12_mean.merge(d12_sum, on=["timestamp", "lat", "lon"])
d12 = d12.merge(d12_max, on=["timestamp", "lat", "lon"])
d12 = d12.merge(d12_last, on=["timestamp", "lat", "lon"])

In [40]:
d12.reset_index(inplace=True)
d12.head()

,timestamp,lat,lon,U2M,T2M,V50M,T10M,SLP,V2M,V10M,U50M,U10M,QV2M,QV10M,BSTAR,GWETTOP,LAI,GWETROOT,GRN,TSURF,EVI,MIR,population density,SGH,elevation,10,11,30,40,60,70,80,90,100,110,120,130,150,160,180,190,200,210,220,agriculture,forest,grassland,wetland,settlement,shrubland,sparse_veg,bare_area,water,PRECTOT,SPEEDMAX,fire_count,dayOfWeek,dayOfYear,sin_dayOfYear,cos_dayOfYear,isWeekend
0,2010-01-02 12:00:00,32.5,-123.750,0.480957,288.75,-7.824219,288.75,102250.843750,-6.039062,-7.039062,0.551758,0.559570,0.010353,0.010239,0.000886,1.0,0.0,1.0,0.0,270.60733,-0.099976,0.799805,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,8.492188,-20.0,5,2,0.034422,0.999407,1
1,2010-01-02 12:00:00,32.5,-123.125,1.187500,288.50,-8.328125,288.50,102198.015625,-6.367188,-7.457031,1.476562,1.390625,0.010269,0.010162,0.000800,1.0,0.0,1.0,0.0,270.60733,-0.099976,0.799805,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,9.210938,-20.0,5,2,0.034422,0.999407,1
2,2010-01-02 12:00:00,32.5,-122.500,1.733398,288.50,-8.648438,288.50,102133.843750,-6.562500,-7.714844,2.195312,2.037109,0.010193,0.010086,0.000740,1.0,0.0,1.0,0.0,270.60733,-0.099976,0.799805,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,9.695312,-20.0,5,2,0.034422,0.999407,1
3,2010-01-02 12:00:00,32.5,-121.875,2.378906,288.00,-8.640625,288.00,102077.015625,-6.351562,-7.515625,3.164062,2.812500,0.009842,0.009766,0.000181,1.0,0.0,1.0,0.0,270.60733,-0.099976,0.799805,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,9.640625,-20.0,5,2,0.034422,0.999407,1
4,2010-01-02 12:00:00,32.5,-121.250,2.876953,287.75,-8.437500,288.00,101999.679688,-6.234375,-7.382812,3.828125,3.404297,0.009506,0.009399,0.000130,1.0,0.0,1.0,0.0,270.60733,-0.099976,0.799805,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,9.773438,-20.0,5,2,0.034422,0.999407,1


In [41]:
d12.to_csv('basegrid_12h.csv', index=False)

In [42]:
del d12
gc.collect()

414

### Daily dataset

In [43]:
d24_mean = (
    d6[to_mean].groupby([pd.Grouper(freq="24H", key="timestamp"), "lat", "lon"]).mean()
)
d24_sum = (
    d6[to_sum].groupby([pd.Grouper(freq="24H", key="timestamp"), "lat", "lon"]).sum()
)
d24_max = (
    d6[to_max].groupby([pd.Grouper(freq="24H", key="timestamp"), "lat", "lon"]).max()
)
d24_last = (
    d6[to_last].groupby([pd.Grouper(freq="24H", key="timestamp"), "lat", "lon"]).last()
)

d24 = d24_mean.merge(d24_sum, on=["timestamp", "lat", "lon"])
d24 = d24.merge(d24_max, on=["timestamp", "lat", "lon"])
d24 = d24.merge(d24_last, on=["timestamp", "lat", "lon"])

In [44]:
d24.reset_index(inplace=True)
d24.head()

,timestamp,lat,lon,U2M,T2M,V50M,T10M,SLP,V2M,V10M,U50M,U10M,QV2M,QV10M,BSTAR,GWETTOP,LAI,GWETROOT,GRN,TSURF,EVI,MIR,population density,SGH,elevation,10,11,30,40,60,70,80,90,100,110,120,130,150,160,180,190,200,210,220,agriculture,forest,grassland,wetland,settlement,shrubland,sparse_veg,bare_area,water,PRECTOT,SPEEDMAX,fire_count,dayOfWeek,dayOfYear,sin_dayOfYear,cos_dayOfYear,isWeekend
0,2010-01-02,32.5,-123.750,0.480957,288.75,-7.824219,288.75,102250.843750,-6.039062,-7.039062,0.551758,0.559570,0.010353,0.010239,0.000886,1.0,0.0,1.0,0.0,270.60733,-0.099976,0.799805,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,8.492188,-20.0,5,2,0.034422,0.999407,1
1,2010-01-02,32.5,-123.125,1.187500,288.50,-8.328125,288.50,102198.015625,-6.367188,-7.457031,1.476562,1.390625,0.010269,0.010162,0.000800,1.0,0.0,1.0,0.0,270.60733,-0.099976,0.799805,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,9.210938,-20.0,5,2,0.034422,0.999407,1
2,2010-01-02,32.5,-122.500,1.733398,288.50,-8.648438,288.50,102133.843750,-6.562500,-7.714844,2.195312,2.037109,0.010193,0.010086,0.000740,1.0,0.0,1.0,0.0,270.60733,-0.099976,0.799805,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,9.695312,-20.0,5,2,0.034422,0.999407,1
3,2010-01-02,32.5,-121.875,2.378906,288.00,-8.640625,288.00,102077.015625,-6.351562,-7.515625,3.164062,2.812500,0.009842,0.009766,0.000181,1.0,0.0,1.0,0.0,270.60733,-0.099976,0.799805,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,9.640625,-20.0,5,2,0.034422,0.999407,1
4,2010-01-02,32.5,-121.250,2.876953,287.75,-8.437500,288.00,101999.679688,-6.234375,-7.382812,3.828125,3.404297,0.009506,0.009399,0.000130,1.0,0.0,1.0,0.0,270.60733,-0.099976,0.799805,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,9.773438,-20.0,5,2,0.034422,0.999407,1


In [45]:
d24.to_csv("basegrid_24h.csv", index=False)